# TensorFlow Classification Example

## Importing the Data

In [1]:
import pandas as pd

In [2]:
census = pd.read_csv('census_data.csv')
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


## EDA using Pandas Profiler

In [4]:
from pandas_profiling import ProfileReport
ProfileReport(census)

Number of variables,14
Number of observations,32561
Total Missing (%),0.0%
Total size in memory,3.5 MiB
Average record size in memory,112.0 B
Numeric,5
Categorical,9
Boolean,0
Date,0
Text (Unique),0
Rejected,0


## Preprocessing

Convert label to Binary

In [5]:
census['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [8]:
# Replacing the string label into binary
census['income_bracket'] = census['income_bracket'].apply(lambda x: 0 if x==' <=50K' else 1)

In [9]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


## Train Test Split

In [10]:
from sklearn.model_selection import train_test_split
x_data = census.drop('income_bracket', axis=1)
y_label = census['income_bracket']
x_train, x_test, y_train, y_test = train_test_split(x_data, y_label, test_size=0.3)

## Feature Columns

In [11]:
import tensorflow as tf

In [12]:
census.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [13]:
# Collecting all columns
allColumns = ['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket']
numericColumns = ['age', 'education_num', 'capital_gain', 
                  'capital_loss', 'hours_per_week']
categoricColumn = ['workclass', 'education', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'native_country']

In [15]:
# Im really proud of this method I made to automate the whole process
# of painfully creating each feature columns and then making a list for DNN
# Make sure that the target column is not in numericColumns and categoricalColumns
def getFeatureColumns(allColumns, numericColumns, categoricalColumns, 
                      categoricalColumnsDimension):
    # Make an empty list of feature columns to be filled iteratively
    feat_cols = []
    for column in allColumns:
        if column in numericColumns:
            feat_cols.append(tf.feature_column.numeric_column(column))
        elif column in categoricalColumns:
            if categoricalColumnsDimension != []:
                dim = categoricalColumnsDimension[categoricalColumns.index(column)]
            else:
                dim = 100
            tempCatCol = tf.feature_column.categorical_column_with_hash_bucket(column,
                                                                        hash_bucket_size=dim)
            # Do this only when using DNNClassr or DNNRegr
            embeddedCatCol = tf.feature_column.embedding_column(tempCatCol, 
                                                               dimension=dim)
            feat_cols.append(embeddedCatCol)
    return feat_cols

In [18]:
feat_cols = getFeatureColumns(allColumns, numericColumns, categoricColumn, [])

## Create input function

In [19]:
input_func = tf.estimator.inputs.pandas_input_fn(x=x_train, y=y_train, batch_size=100,
                                                num_epochs=None, shuffle=True)

## Create model object

In [23]:
model = tf.estimator.DNNClassifier(hidden_units=[15, 15, 15, 15], feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpk3qqgfj_', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f83104e3278>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Training the DNN Classifier

In [28]:
model.train(input_fn=input_func, max_steps=1100)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpk3qqgfj_/model.ckpt-1001
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1001 into /tmp/tmpk3qqgfj_/model.ckpt.
INFO:tensorflow:loss = 38.23356, step = 1002
INFO:tensorflow:Saving checkpoints for 1100 into /tmp/tmpk3qqgfj_/model.ckpt.
INFO:tensorflow:Loss for final step: 38.456757.


## Evaluating against TestSet

In [29]:
test_input_funct = tf.estimator.inputs.pandas_input_fn(x=x_test, y=y_test, batch_size=10,
                                                      num_epochs=None, shuffle=False)

In [30]:
model.evaluate(input_fn=test_input_funct, steps=1)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-20T16:25:43Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpk3qqgfj_/model.ckpt-1100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2019-06-20-16:25:45
INFO:tensorflow:Saving dict for global step 1100: accuracy = 0.9, accuracy_baseline = 0.8, auc = 0.93749994, auc_precision_recall = 0.7916663, average_loss = 0.2934205, global_step = 1100, label/mean = 0.2, loss = 2.9342048, precision = 1.0, prediction/mean = 0.1507503, recall = 0.5
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1100: /tmp/tmpk3qqgfj_/model.ckpt-1100


{'accuracy': 0.9,
 'accuracy_baseline': 0.8,
 'auc': 0.93749994,
 'auc_precision_recall': 0.7916663,
 'average_loss': 0.2934205,
 'label/mean': 0.2,
 'loss': 2.9342048,
 'precision': 1.0,
 'prediction/mean': 0.1507503,
 'recall': 0.5,
 'global_step': 1100}